In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pytorch_lightning import seed_everything, LightningModule, Trainer
from sklearn.utils import class_weight
import torch.nn as nn
import torch
from pytorch_lightning.callbacks import EarlyStopping,ModelCheckpoint,LearningRateMonitor
from torch.optim.lr_scheduler import CyclicLR, ReduceLROnPlateau,CosineAnnealingWarmRestarts,OneCycleLR,CosineAnnealingLR
import torchvision
from sklearn.metrics import classification_report,f1_score,accuracy_score
from PIL import Image
from torch.utils.data import DataLoader, Dataset,random_split
import timm
import torchmetrics
import torchvision.models as models

In [2]:
img_size=224
import albumentations as A
from albumentations.pytorch import ToTensorV2
aug= A.Compose([
            A.Resize(img_size+32,img_size+32),
            A.RandomCrop(img_size,img_size),
            A.HorizontalFlip(0.5),
            A.VerticalFlip(0.5),
            A.ShiftScaleRotate(rotate_limit=3),
            A.Blur(),A.RandomGamma(),
            A.Sharpen(), A.GaussNoise(),
            A.CoarseDropout(8,64,64),
            A.CLAHE(0.5),
            A.Normalize(),
            ToTensorV2(p=1.0),
        ], p=1.0)

In [3]:
class DataReader(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, index):
        x=self.dataset[index][0]
        y=self.dataset[index][1]
        if self.transform:
            x=np.array(x)
            x=self.transform(image=x)['image']
        return x, y
    
    def __len__(self):
        return len(self.dataset)

In [4]:
class OurModel(LightningModule):
    def __init__(self):
        super(OurModel,self).__init__()
        #architecute
        self.model =  timm.create_model(model_name,pretrained=True)
        self.fc1=nn.Linear(1000,500)
        self.relu=nn.ReLU()
        self.fc2= nn.Linear(500,5)
 
        #parameters
        self.lr=1e-3
        self.batch_size=72
        self.numworker=12
        self.acc = torchmetrics.Accuracy()
        self.criterion=nn.CrossEntropyLoss()
        #LabelSmoothing()
        
        self.trainacc,self.valacc=[],[]
        self.trainloss,self.valloss=[],[]
        
        self.scheduler='constant'
        
        self.dataset=torchvision.datasets.ImageFolder('dataset')
        
        self.train_set, self.val_set =random_split(self.dataset, 
                            [int(len(self.dataset)*0.8), int(len(self.dataset)*0.2)])
    def forward(self,x):
        x= self.model(x)
        x=self.fc1(x)
        x=self.relu(x)
        x=self.fc2(x)
        return x

    def configure_optimizers(self):
        opt=torch.optim.AdamW(params=self.parameters(),lr=self.lr )
        if self.scheduler=='cosine':
            scheduler=CosineAnnealingLR(opt,T_max=10,  eta_min=1e-6, last_epoch=-1)
            return {'optimizer': opt,'lr_scheduler':scheduler}
        elif self.scheduler=='reduce':
            scheduler=ReduceLROnPlateau(opt,mode='min', factor=0.5, patience=5)
            return {'optimizer': opt,'lr_scheduler':scheduler,'monitor':'val_loss'}
        elif self.scheduler=='warm':
            scheduler=CosineAnnealingWarmRestarts(opt,T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1)
            return {'optimizer': opt,'lr_scheduler':scheduler}
        elif self.scheduler=='constant':
            return opt
 
    def train_dataloader(self):
        return DataLoader(DataReader(self.train_set,aug), batch_size = self.batch_size, 
                          num_workers=self.numworker,
                          pin_memory=True,shuffle=False)

    def training_step(self,batch,batch_idx):
        image,label=batch
        out = self(image)
        loss=self.criterion(out,label)
        acc=self.acc(out,label)
        return {'loss':loss,'acc':acc}

    def training_epoch_end(self, outputs):
        loss=torch.stack([x["loss"] for x in outputs]).mean().detach().cpu().numpy().round(2)
        acc=torch.stack([x["acc"] for x in outputs]).mean().detach().cpu().numpy().round(2)
        self.trainacc.append(acc)
        self.trainloss.append(loss)
        #print('training loss accuracy ',self.current_epoch,loss, acc)
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        
    def val_dataloader(self):
        ds=DataLoader(DataReader(self.val_set,aug), batch_size = self.batch_size,
                      num_workers=self.numworker,pin_memory=True, shuffle=False)
        return ds

    def validation_step(self,batch,batch_idx):
        image,label=batch
        out=self(image)
        loss=self.criterion(out,label)
        acc=self.acc(out,label)
        return {'loss':loss,'acc':acc}

    def validation_epoch_end(self, outputs):
        loss=torch.stack([x["loss"] for x in outputs]).mean().detach().cpu().numpy().round(2)
        acc=torch.stack([x["acc"] for x in outputs]).mean().detach().cpu().numpy().round(2)
        self.valacc.append(acc)
        self.valloss.append(loss)
        print('validation loss accuracy ',self.current_epoch,loss, acc)
        self.log('val_loss', loss)
        self.log('val_acc', acc)

In [5]:
model_name='resnest50d'
model=OurModel()

In [6]:
lr_monitor = LearningRateMonitor(logging_interval='epoch')
trainer = Trainer(max_epochs=50, auto_lr_find=False, auto_scale_batch_size=False,
                deterministic=True,
                gpus=-1,precision=16,
                accumulate_grad_batches=4,
                stochastic_weight_avg=False,
                enable_progress_bar = False,
                num_sanity_val_steps=0,
                callbacks=[lr_monitor]
                 )

Using 16bit native Automatic Mixed Precision (AMP)
[W Context.cpp:69] Warning: torch.set_deterministic is in beta, and its design and  functionality may change in the future. (function operator())
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2021-12-06 16:08:30.354510: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | ResNet           | 27.5 M
1 | fc1       | Linear           | 500 K 
2 | relu      | ReLU             | 0     
3 | fc2       | Linear           | 2.5 K 
4 | acc       | Accuracy         | 0     
5 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
28.0 M    Trainable params
0         Non-trainable params
28.0 M    Total params
55.972    Total estimated model params size (MB)


validation loss accuracy  0 1.97 0.3
validation loss accuracy  1 1.42 0.4
validation loss accuracy  2 1.25 0.45
